# Setup

In [ ]:
!pip install pythainlp==2.2.6

     |████████████████████████████████| 10.6 MB 5.2 MB/s 
     |████████████████████████████████| 743 kB 56.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires typing-extensions~=3.7.4, but you have typing-extensions 3.10.0.2 which is incompatible.


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!pip install .
%cd /content

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.22 MiB | 33.82 MiB/s, done.
Resolving deltas: 100% (2417/2417), done.
/content/fastText
Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.7.1-py2.py3-none-any.whl (200 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3089883 sha256=67e7a8c29a23a05ff001b896dd5feee2c8692c1cf6f0855e18ed61f7f4f0d0c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-60w99hop/wheels/22/04/6e/b3aba25c1a5

# Get Data and Clean

In [1]:
!unzip wongnai-dataset.zip

Archive:  wongnai-dataset.zip
 extracting: sample_submission.csv   
  inflating: test_file.csv           
  inflating: w_review_train.csv      
   creating: __MACOSX/
  inflating: __MACOSX/._sample_submission.csv  
  inflating: __MACOSX/._test_file.csv  
  inflating: __MACOSX/._w_review_train.csv  


In [ ]:
with open("/content/w_review_train.csv") as f:
    lines = f.readlines()

In [ ]:
reviews_long_str = " ".join(lines)

In [ ]:
reviews_long_lst = reviews_long_str.split(";")

In [ ]:
import numpy as np
import re
import pandas as pd
from pythainlp import word_tokenize

In [ ]:
texts = []
stars = []

for i in np.arange(0, len(reviews_long_lst)-1):
    if len(reviews_long_lst[i+1]) >= 1: # exclude empty reviews
        if reviews_long_lst[i+1][0] in set(['1','2','3','4','5']): # some reviews don't have score, check first if score is given
            text = reviews_long_lst[i]

            # clean text
            if i > 0:
                text = text[1:]
            text = re.sub("\"", "", text)
            text = re.sub("\n", " ", text)
            text = text.strip()

            # store for make dataframe
            texts.append(text)
            stars.append(int(reviews_long_lst[i+1][0]))

In [ ]:
df = pd.DataFrame({
    "review":texts,
    "star":stars
})

In [ ]:
df.head()

,review,star
0,ร้านอาหารใหญ่มากกกกกกก เลี้ยวเข้ามาเจอห้องน้...,3
1,อาหารที่นี่เป็นอาหารจีนแคะที่หากินยากในบ้านเรา...,4
2,ปอเปี๊ยะสด ทุกวันนี้รู้สึกว่าหากินยาก (ร้านที่...,3
3,รัานคัพเค้กในเมืองไทยมีไม่มาก หลายๆคนอาจจะสงสั...,5
4,อร่อย!!! เดินผ่านDigital gatewayทุกวัน ไม่ยักร...,5


In [ ]:
df.tail()

,review,star
39996,รู้จักร้านนี้จากวงใน ร้านอยู่ในอาคารพาณิชย์ตรง...,4
39997,ร้านซูชิอาหารญี่ปุ่น อยู่ตรงสะพานลอย เกษตรประต...,4
39998,Cantina Wine Bar & Italian Kitchen ร้านเล็กๆ อ...,5
39999,ร้านเค้กน่ารักๆ ตรงชั้นล่างของห้างเซ็นทรัลลาดพ...,3
40000,วันนี้มากินกันไกลถึงแม่สอดจังหวัดตากติดกับชายแ...,3


# Write data to text file for fasttext

In [ ]:
lines_new = []
for i, row in df.iterrows():
    line = "__label__" + str(row["star"]) + " " + " ".join(word_tokenize(row["review"]))
    lines_new.append(line)

In [ ]:
print(lines_new[0:5])

['__label__3 ร้านอาหาร ใหญ่ มาก กก กก กก     เลี้ยว เข้ามา เจอ ห้องน้ำ ก่อน เลย   เออ แปลก ดี     ห้อง ทาน หลัก ๆ อยู่ ชั้น   2   มี กาแฟ   น้ำผึ้ง   ซึ่ง ก็ แค่ เอา น้ำผึ้ง มา ราด   แพง เวอร์   อย่า สั่ง เลย     ลาบ ไข่ต้ม   ไข่ มัน คาว อะ   เลย ไม่ ประทับใจ เท่าไหร่    ทอดมัน หัวปลี กรอบ อร่อย ต้อง เบิ้ล     พะแนง ห่อ ไข่ อร่อย ดี   เห้ย แต่ ราคา   150 บาท มัน เกินไป นะ   รับ ไม่ ไหว ว    เลิก กิน แล้ว มี ขนมหวาน ให้ กิน ฟรี เล็กน้อย   ) ขนมไทย )       คง ไม่ ไป ซ้ำ   แพง เกิน', '__label__4 อาหาร ที่นี่ เป็น อาหารจีน แคะ ที่ หากิน ยาก ใน บ้าน เรา   ตัว ร้าน ตั้งอยู่ ที่ ถนน พุทธมณฑล สาย   3   ไป ตาม   ถ. ปิ่น เกล้า - นครชัยศรี   เมื่อ ถึง พุทธมณฑล สาย   3   ก็ เลี้ยว เข้าไป ประมาณ   500   เมตร   ร้าน อยู่ ทาง ซ้ายมือ ค่ะ   มี คน บอก มา ว่าความ พิเศษ ของ ร้าน นี้ คือ กุ๊ก เก่า และ เป็น กุ๊ก รุ่น สุดท้าย จาก   ฮก   ลก   ซิ่ว ”   ภัตตาคาร จีน ชื่อดัง ย่าน ราชประสงค์   ที่ เลิกกิจการ ไป แล้ว   ต้อง คน ที่ อายุ เลข   5   ขึ้นไป จึง จะ เคย กิน   ฮก   ลก   ซิ่ว    จาน เด็ด ที่ มี ขาย ที่นี่

In [ ]:
lines_new_train = lines_new[0:35000]
lines_new_test = lines_new[35000:]

In [ ]:
with open("wongnai.train", "w") as f:
    for line in lines_new_train:
        f.write(line + '\n')

with open("wongnai.test", "w") as f:
    for line in lines_new_test:
        f.write(line + '\n')

In [ ]:
!head wongnai.train

__label__3 ร้านอาหาร ใหญ่ มาก กก กก กก     เลี้ยว เข้ามา เจอ ห้องน้ำ ก่อน เลย   เออ แปลก ดี     ห้อง ทาน หลัก ๆ อยู่ ชั้น   2   มี กาแฟ   น้ำผึ้ง   ซึ่ง ก็ แค่ เอา น้ำผึ้ง มา ราด   แพง เวอร์   อย่า สั่ง เลย     ลาบ ไข่ต้ม   ไข่ มัน คาว อะ   เลย ไม่ ประทับใจ เท่าไหร่    ทอดมัน หัวปลี กรอบ อร่อย ต้อง เบิ้ล     พะแนง ห่อ ไข่ อร่อย ดี   เห้ย แต่ ราคา   150 บาท มัน เกินไป นะ   รับ ไม่ ไหว ว    เลิก กิน แล้ว มี ขนมหวาน ให้ กิน ฟรี เล็กน้อย   ) ขนมไทย )       คง ไม่ ไป ซ้ำ   แพง เกิน
__label__4 อาหาร ที่นี่ เป็น อาหารจีน แคะ ที่ หากิน ยาก ใน บ้าน เรา   ตัว ร้าน ตั้งอยู่ ที่ ถนน พุทธมณฑล สาย   3   ไป ตาม   ถ. ปิ่น เกล้า - นครชัยศรี   เมื่อ ถึง พุทธมณฑล สาย   3   ก็ เลี้ยว เข้าไป ประมาณ   500   เมตร   ร้าน อยู่ ทาง ซ้ายมือ ค่ะ   มี คน บอก มา ว่าความ พิเศษ ของ ร้าน นี้ คือ กุ๊ก เก่า และ เป็น กุ๊ก รุ่น สุดท้าย จาก   ฮก   ลก   ซิ่ว ”   ภัตตาคาร จีน ชื่อดัง ย่าน ราชประสงค์   ที่ เลิกกิจการ ไป แล้ว   ต้อง คน ที่ อายุ เลข   5   ขึ้นไป จึง จะ เคย กิน   ฮก   ลก   ซิ่ว    จาน เด็ด ที่ มี ขาย ที่นี่ แห่ง

In [ ]:
!head wongnai.test

__label__4 ร้าน นี้ ตั้งอยู่ ริมถนน ลาดพร้าว - วัง หิน ค่ะ   เป็น ร้าน เล็ก ๆ ค่ะ    ร้าน นี้ คน ค่อนข้าง เยอะ   ต้อง ต่อ คิว นะคะ   โต๊ะ มี ไม่ เยอะ เลย ต้อง รอ คิว นิดนึง   เมนู เด็ด ร้า นี้ คือ   หมู ชีส    คือ มัน เด็ด มาก หมู นุ่ม ๆ กับ ชีส เยิ้ม ๆ   มัน ช่าง เข้ากัน จริงๆ   ฟิน!!!!!    น้ำจิ้ม รสเด็ด   กิน กับ ชาบู เข้า ก๊าน เข้ากัน   อร่อย ลื้มมมม    นอกจาก มี ยังมี เซ ล ม่อน   ซูชิ   ไอติม อีกด้วย
__label__4 วันนี้ มีโอกาส มา งาน ซ้อม รับปริญญา ที่ ม. สวน ดุสิต   ก่อน กลับ เลย ไม่ พลาด ที่จะ แวะ เข้าไป ร้าน   Home   Bakery   สักหน่อย   ปกติ เคย ทาน แต่ ร้าน ข้างนอก ที่ รับ มา อีกที       ร้าน อยู่   zone   ด้านหน้า ทางเข้า มหาลัย   เดิน เข้าไป เจอ   Bakery   หลาย หลาย มาก   ทั้ง ขนม ฮิต สุด ดัง จาก   Toffee   Cake   ที่ มี ทั้ง แบบ   Original ,   หน้า   Almond ,   หน้า   Macadamia ,   ฯลฯ   รวมทั้ง มี หลาย ขนาด ให้ เลือก ซื้อ อีก ต่างหาก   ยัง ไม่ รวม   Bakery   ชนิด อื่นๆ   เค้ก ,   แยมโรล ,   ครัวซอง   ขนมปัง โรล ไส้ ต่างๆ   พวกเรา ก็ หยิบ กัน เพลิน   ได้ เป็น ถุง ใหญ่ ๆ กลับ

In [ ]:
import fasttext

In [ ]:
model = fasttext.train_supervised(input="wongnai.train")

In [ ]:
model.test("wongnai.test") # output is (how many, precision, recall)

(5001, 0.5604879024195161, 0.5604879024195161)

In [ ]:
# predict a new review
model.predict(" ".join(word_tokenize("อร่อยมากต้องไปกินอีกให้ได้")))

(('__label__5',), array([0.95409226]))

In [ ]:
# predict a new review
model.predict(" ".join(word_tokenize("ไม่ได้เรื่องแพงก็แพงไม่ไปกินอีกแล้ว")))

(('__label__2',), array([0.43817481]))